In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from datetime import datetime  # 현재 날짜를 가져오기 위해 추가
import mysql.connector  # MySQL 연결을 위해 추가
import time

# MySQL 서버에 연결
conn = mysql.connector.connect(
    host='13.211.174.98',      # 호스트 이름
    user='root',       # MySQL 사용자 이름
    password='hanagift',   # MySQL 사용자 비밀번호
    database='db'  # 연결할 데이터베이스 이름
)

# 커서 생성
cursor = conn.cursor()


data_list = [] 
# 웹 드라이버 설정 (예: Chrome)
driver = webdriver.Chrome()

# 1. 페이지 로드
url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_07i.do#//HanaBank"
driver.get(url)

# 2. 페이지 로드 대기
wait = WebDriverWait(driver, 10)

# 3. 로딩 레이어가 사라질 때까지 대기
try:
    loading_layer = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#OPB_loadingLayerID_generatedByJSOPB_modalMaskID_generatedByJS")))
    wait.until(EC.invisibility_of_element(loading_layer))
except TimeoutException:
    print("Loading layer did not disappear, continuing...")

# 4. "기간환율변동" 선택 클릭
try:
    period_exchange_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='inqType_p'].radioForm")))
    period_exchange_button.click()
except (TimeoutException, ElementClickInterceptedException):
    driver.execute_script("document.querySelector(\"label[for='inqType_p']\").click();")

# 5. 날짜 수정 (조회 기간 시작일)
today = datetime.now().strftime('%Y-%m-%d')
date_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tmpInqStrDt_p")))
date_input.clear()
date_input.send_keys(today)

# 6. 통화 선택 (코드 테이블에 있는 외화 목록을 반복 조회)
currency_codes = ['AUD', 'BRL', 'CAD', 'CNY', 'EUR', 'GBP', 'JPY', 'MYR', 'RUB', 'SEK', 'USD', 'VND']  # 테이블에서 확인한 코드들

for currency_code in currency_codes:
    # 통화 선택 (통화별로 반복)
    currency_select = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#curCd")))
    select = Select(currency_select)
    select.select_by_value(currency_code)

    # 7. "조회" 버튼 클릭
    query_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
    query_button.click()

    # 8. 조회 후 5초 대기
    time.sleep(5)

    # 9. 테이블 데이터 크롤링
    rows = driver.find_elements(By.CSS_SELECTOR, "#searchContentDiv > div.printdiv > table > tbody > tr")

    for row in rows:
        columns = row.find_elements(By.TAG_NAME, "td")
        data = [column.text for column in columns]
        
        # 첫 번째 값 제외, 나머지를 float로 변환
        for i in range(1, len(data)):
            try:
                data[i] = float(data[i].replace(',', ''))  # 쉼표 제거 후 float 변환
            except ValueError:
                print(f"변환 실패: {data[i]}")

        # 크롤링한 데이터 추가 (통화 코드와 현재 날짜 포함)
        data.append(currency_code)  # 통화 코드를 추가
        data_list.append(data)

# 웹 드라이버 종료
driver.quit()

# 10. MySQL에 데이터를 일괄 삽입
insert_query = """
INSERT INTO exchangeRate (exchangeDate, buyCash, sellCash, sendRemit, receiveRemit, buyForeignCheck, exchangeRate, previousComparison, rateGap, usdConversionRate, currencyCode)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# MySQL에 일괄 삽입 (executemany 사용)
try:
    cursor.executemany(insert_query, data_list)
    conn.commit()  # 모든 데이터를 한 번에 커밋
    print(f"{len(data_list)} 개의 데이터가 성공적으로 삽입되었습니다.")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    conn.rollback()  # 오류 발생 시 롤백

# MySQL 연결 종료
cursor.close()
conn.close()

12 개의 데이터가 성공적으로 삽입되었습니다.
